In [1]:
import string
from itertools import permutations

In [2]:
X_RANGE = 20
Y_RANGE = 20
DIRECTIONS = ["N", "S", "E", "W"]
STR_COST = 10
ROT_COST = 50

In [3]:
def get_x_coordinate(coordinate): #get int x-coordinate from string coordinate
    x_coordinate = string.ascii_uppercase.index(coordinate[0])
    return x_coordinate

def get_y_coordinate(coordinate): #get int y-coordinate from string coordinate
    y_coordinate = int(coordinate[2:-2])
    return y_coordinate

def get_dir(coordinate): #get direction from string coordinate
    dir_coordinate = coordinate[-1:]
    return dir_coordinate

def get_alphanumeric_coordinates(x, y, dir): #convert to alphanumeric coordinates
    alphanumeric_coordinate = '{alphabet}-{number}-{dir}'.format(
        alphabet=string.ascii_uppercase[x],
        number=y,
        dir=dir)
    return alphanumeric_coordinate

In [4]:
def check_within_bounds(coordinate): #check that robot is within 20x20 arena when camera is at a coordinate
    
    dir = get_dir(coordinate)
    x = get_x_coordinate(coordinate)
    y = get_y_coordinate(coordinate)
    
    if dir == 'N':
        if x >= 1 and x < (X_RANGE - 1) and y >= 2 and y < Y_RANGE:
            return True
    elif dir == 'S':
        if x >= 1 and x < (X_RANGE - 1) and y >= 0 and y < (Y_RANGE - 2):
            return True
    elif dir == 'W':
        if x >= 0 and x < (X_RANGE - 2) and y >= 1 and y < (Y_RANGE - 1):
            return True
    elif dir == 'E':
        if x >= 2 and x < X_RANGE and y >= 1 and y < (Y_RANGE - 1):
            return True
    else:
        return False

def get_virtual_boundary(obstacles): #add virtual boundary of 50x50cm surrounding each obstacle

    boundary = []
    
    for obstacle in obstacles:
        
        x_coordinate = get_x_coordinate(obstacle)
        y_coordinate = get_y_coordinate(obstacle)

        left_bounds = x_coordinate - 2
        right_bounds = x_coordinate + 2
        top_bounds = y_coordinate + 2
        bottom_bounds = y_coordinate - 2
        
        for i in range(left_bounds, right_bounds+1):
            for j in range(bottom_bounds, top_bounds+1):
                for dir in DIRECTIONS:
                    coordinate = get_alphanumeric_coordinates(i, j, dir)
                    if check_within_bounds(coordinate):
                        boundary.append(coordinate)
        
    return boundary

def check_valid_node(node, obstacles): #camera can only be at coordinate where i
    boundaries = get_virtual_boundary(obstacles)
    if check_within_bounds(node) and (node not in boundaries):
        return True
    else:
        return False

In [5]:
def get_coordinates(obstacles):
    coordinates = {}
    
    for i in range(X_RANGE):
        for j in range(Y_RANGE):
            for dir in DIRECTIONS:
                coordinate = get_alphanumeric_coordinates(i, j, dir)
                if check_valid_node(coordinate, obstacles):
                    coordinates[get_alphanumeric_coordinates(i, j, dir)] = (i, j, dir)
                    
    return coordinates

In [6]:
def get_goals(obstacles):
    goals = []
    
    for obstacle in obstacles:
        dir = get_dir(obstacle)
        x = get_x_coordinate(obstacle)
        y = get_y_coordinate(obstacle)
        
        #destination is 3 grids away in the opposite direction of the obstacle
        if dir == "N":
            goal = get_alphanumeric_coordinates(x, y + 3, "S")
        elif dir == "S":
            goal = get_alphanumeric_coordinates(x, y - 3, "N")
        elif dir == "E":
            goal = get_alphanumeric_coordinates(x + 3, y, "W")
        elif dir == "W":
            goal = get_alphanumeric_coordinates(x - 3, y, "E")
        goals.append(goal)
        
    return goals

In [7]:
def add_valid_adjacent_nodes(node_f, node_b, node_fc, node_fa, node_bc, node_ba, obstacles):
    adjacent_nodes = []

    if check_valid_node(node_f, obstacles):
        adjacent_nodes.append((node_f, STR_COST, 'F')) #tuple: coordinate, cost, action
    if check_valid_node(node_b, obstacles):
        adjacent_nodes.append((node_b, STR_COST, 'B'))
    if check_valid_node(node_fc, obstacles):
        adjacent_nodes.append((node_fc, ROT_COST, 'FC'))
    if check_valid_node(node_fa, obstacles):
        adjacent_nodes.append((node_fa, ROT_COST, 'FA'))
    if check_valid_node(node_bc, obstacles):
        adjacent_nodes.append((node_bc, ROT_COST, 'BC'))
    if check_valid_node(node_ba, obstacles):
        adjacent_nodes.append((node_ba, ROT_COST, 'BA'))
    
    return adjacent_nodes


def get_adjacent_nodes(node, obstacles):

    dir = get_dir(node)
    x = get_x_coordinate(node)
    y = get_y_coordinate(node)
            
    if dir == 'N':
        node_f = get_alphanumeric_coordinates(x, y + 1, 'N')
        node_b = get_alphanumeric_coordinates(x, y - 1, 'N')
        node_fc = get_alphanumeric_coordinates(x + 4, y, 'E')
        node_fa = get_alphanumeric_coordinates(x - 4, y , 'W')
        node_bc = get_alphanumeric_coordinates(x, y - 4, 'E')
        node_ba = get_alphanumeric_coordinates(x, y - 4, 'W')
        
        return add_valid_adjacent_nodes(node_f, node_b, node_fc, node_fa, node_bc, node_ba, obstacles)
    
    if dir == 'S':
        node_f = get_alphanumeric_coordinates(x, y - 1, 'S')
        node_b = get_alphanumeric_coordinates(x, y + 1, 'S')
        node_fc = get_alphanumeric_coordinates(x - 4, y , 'W')
        node_fa = get_alphanumeric_coordinates(x + 4, y, 'E')
        node_bc = get_alphanumeric_coordinates(x, y + 4, 'W')
        node_ba = get_alphanumeric_coordinates(x, y + 4, 'E')
        
        return add_valid_adjacent_nodes(node_f, node_b, node_fc, node_fa, node_bc, node_ba, obstacles)
                
    if dir == 'W':
        node_f = get_alphanumeric_coordinates(x - 1, y, 'W')
        node_b = get_alphanumeric_coordinates(x + 1, y, 'W')
        node_fc = get_alphanumeric_coordinates(x, y + 4, 'N')
        node_fa = get_alphanumeric_coordinates(x, y - 4, 'S')
        node_bc = get_alphanumeric_coordinates(x + 4, y, 'N')
        node_ba = get_alphanumeric_coordinates(x + 4, y, 'S' )
        
        return add_valid_adjacent_nodes(node_f, node_b, node_fc, node_fa, node_bc, node_ba, obstacles)   

    if dir == 'E':
        node_f = get_alphanumeric_coordinates(x + 1, y, 'E')
        node_b = get_alphanumeric_coordinates(x - 1, y, 'E')
        node_fc = get_alphanumeric_coordinates(x, y - 4, 'S')
        node_fa = get_alphanumeric_coordinates(x, y + 4, 'N')   
        node_bc = get_alphanumeric_coordinates(x - 4 , y, 'S')
        node_ba = get_alphanumeric_coordinates(x - 4, y, 'N')
        
        return add_valid_adjacent_nodes(node_f, node_b, node_fc, node_fa, node_bc, node_ba, obstacles)
    

In [8]:
def create_graph(obstacles): #adjacency list
    graph = {}
    
    coordinates = get_coordinates(obstacles).keys()
    for coordinate in coordinates:
        graph[coordinate] = get_adjacent_nodes(coordinate, obstacles)
    
    return graph

def permutate_goals(obstacles):
    goals = get_goals(obstacles)
    permutated_goals = permutations(goals)
    return [list(p) for p in permutated_goals]